# Giriş işlemleri

In [ ]:
%%capture
!pip install pandas numpy datasets

In [ ]:
import os
import pandas as pd
import numpy as np
from datasets import load_dataset
import sys

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/TurkNLP-ModelBench"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
sys.path.append(kok_dizin)
from dizin_yardimcisi import veri_kumesi_dizin_al

# Veri kümesi ön işlemleri

## Veri Kümesi İçin Kök Klasör Oluştur 

In [ ]:
# Veri kümesi dosyalarının kaydedileceği dizin
veri_kumeleri_dizini = veri_kumesi_dizin_al()
os.makedirs(veri_kumeleri_dizini, exist_ok=True)
print(f"Veri kümeleri dizini: {veri_kumeleri_dizini}")

## Veri kümeleri ve parçalama fonksiyonları

In [ ]:
def veri_kumesi_yukle(koleksiyon_adi, veri_kumesi_adi):
    """
    Args:
    koleksiyon_adi (str): Veri kümesinin bulunduğu koleksiyon adı
    veri_kumesi_adi (str): Yüklenecek veri kümesinin adı

    Returns:
    veri_kumesi (datasets.Dataset): Yüklenen veri kümesi

    """
    print(f"Veri kümesi yükleniyor: {veri_kumesi_adi}")
    veri_kumesi = load_dataset(koleksiyon_adi, veri_kumesi_adi)
    print(f"Veri kümesi yüklendi: {veri_kumesi_adi}")
    return veri_kumesi
def boyut_kontrol(veri_kumesi, veri_kumesi_parca_boyutu) -> bool:
    """
    Veri kümesinin boyutunu kontrol eder.
    
    Args:
    veri_kumesi (datasets.Dataset): Kontrol edilecek veri kümesi
    veri_kumesi_parca_boyutu (int): veri kümesinden istenilen parçanın boyutu
    
    Returns:
    bool: Veri kümesi belirtilen boyuttan büyükse True, aksi halde False
    """
    if veri_kumesi_parca_boyutu is not None and len(veri_kumesi) < veri_kumesi_parca_boyutu:
        return False
    else:
        return True
def veri_kumesi_parcala(veri_kumesi, egitim_boyutu=None, sinama_boyutu=None, tohum_degeri=42):
    """
    Veri kümesini belirli büyüklükteki eğitim ve test alt kümelerine ayırır.
    
    Parametreler:
        veri_kumesi: Parçalanacak veri kümesi
        egitim_boyutu: Eğitim kümesi büyüklüğü (None ise tamamını kullanır)
        sinama_boyutu: Test kümesi büyüklüğü (None ise tamamını kullanır)
        tohum_degeri: Rastgele seçim için kullanılacak tohum değeri
        
    Dönüş:
        egitim_kumesi, sinama_kumesi, sayilar_dict: Eğitim ve test veri kümeleri ve etiket sayıları
    """
    
    rng = np.random.RandomState(tohum_degeri)
    
    # Eğitim ve test kümelerini al
    egitim_kumesi = veri_kumesi["train"]
    sinama_kumesi = veri_kumesi["test"]
    
    sayilar_dict = {'egitim': {}, 'sinama': {}}
    
    # Eğitim kümesi işleme
    egitim_kumesi, egitim_sayilari = alt_kume_olustur(egitim_kumesi, egitim_boyutu, rng)
    sayilar_dict['egitim'] = egitim_sayilari
    
    # Test kümesi işleme
    sinama_kumesi, sinama_sayilari = alt_kume_olustur(sinama_kumesi, sinama_boyutu, rng)
    sayilar_dict['sinama'] = sinama_sayilari
    
    return egitim_kumesi, sinama_kumesi, sayilar_dict

def alt_kume_olustur(veri_kumesi, boyut, rng):
    """
    Veri kümesinden istenen boyutta dengeli bir alt küme oluşturur.
    
    Parametreler:
        veri_kumesi: İşlenecek veri kümesi
        boyut: İstenen toplam örnek sayısı (None ise tamamını kullanır)
        rng: Rastgele sayı üretici
        
    Dönüş:
        alt_kume, etiket_sayilari: Alt küme ve etiketlerin sayıları
    """
    # Etiketleri al
    etiketler = np.array(veri_kumesi['label'])
    
    # Mevcut benzersiz etiketleri bul
    benzersiz_etiketler = np.unique(etiketler)
    
    # 0 ve 1 etiketleri için indeksleri al (bu veri kümelerinin sadece 0 ve 1 etiketi var)
    indeksler_0 = np.where(etiketler == 0)[0] if 0 in benzersiz_etiketler else np.array([])
    indeksler_1 = np.where(etiketler == 1)[0] if 1 in benzersiz_etiketler else np.array([])
    diger_indeksler = np.where((etiketler != 0) & (etiketler != 1))[0]
    
    # Her bir etiketin sayısı
    sayi_0 = len(indeksler_0)
    sayi_1 = len(indeksler_1)
    sayi_diger = len(diger_indeksler)
    
    # Her bir etiketten alınacak örnek sayısı
    if boyut is None:
        # İstenen boyut verilmemişse, mevcut verilerin tamamını al
        num_0 = sayi_0
        num_1 = sayi_1
        num_diger = sayi_diger
    else:
        # İki ana sınıftan (0 ve 1) eşit sayıda örnek almaya çalış
        # Toplam boyut ikiye bölünmeli
        if sayi_0 > 0 and sayi_1 > 0:
            # Her iki etiketten de alınabilecek maksimum eşit sayı
            hedef_etiket_sayisi = boyut // 2
            
            # Her sınıftan en fazla mevcut olan kadar alabiliriz
            num_0 = min(sayi_0, hedef_etiket_sayisi)
            num_1 = min(sayi_1, hedef_etiket_sayisi)
            
            # Hedeflenen toplam örneklerden kalan boşluk
            kalan_boyut = boyut - (num_0 + num_1)
            
            # Kalan boşluğu doldur (önce eksik kalan etiketler için)
            if kalan_boyut > 0:
                if sayi_0 > num_0:  # 0 etiketinden daha fazla alabilir miyiz?
                    ek_0 = min(sayi_0 - num_0, kalan_boyut)
                    num_0 += ek_0
                    kalan_boyut -= ek_0
                    
                if kalan_boyut > 0 and sayi_1 > num_1:  # 1 etiketinden daha fazla alabilir miyiz?
                    ek_1 = min(sayi_1 - num_1, kalan_boyut)
                    num_1 += ek_1
                    kalan_boyut -= ek_1
                    
                # Hala kalan boşluk varsa diğer etiketlerden doldur
                num_diger = min(kalan_boyut, sayi_diger)
            else:
                num_diger = 0
        else:
            # Eğer bir etiket hiç yoksa, diğerinden mümkün olduğu kadar al
            if sayi_0 > 0:
                num_0 = min(sayi_0, boyut)
                num_1 = 0
            elif sayi_1 > 0:
                num_0 = 0
                num_1 = min(sayi_1, boyut)
            else:
                num_0 = 0
                num_1 = 0
            
            # Kalan boşluğu diğer etiketlerden doldur
            num_diger = min(boyut - (num_0 + num_1), sayi_diger)
    
    # Rastgele indeksleri seç
    secilen_indeksler = []
    
    if num_0 > 0:
        secilen_indeksler.extend(rng.choice(indeksler_0, size=num_0, replace=False))
    if num_1 > 0:
        secilen_indeksler.extend(rng.choice(indeksler_1, size=num_1, replace=False))
    if num_diger > 0:
        secilen_indeksler.extend(rng.choice(diger_indeksler, size=num_diger, replace=False))
    
    # İndeksleri karıştır 
    rng.shuffle(secilen_indeksler)
    
    # Alt kümeyi oluştur
    alt_kume = veri_kumesi.select(secilen_indeksler)
    
    # Etiketlerin sayısını hesapla
    alt_etiketler = np.array(alt_kume['label'])
    unique, counts = np.unique(alt_etiketler, return_counts=True)
    
    # Etiket sayılarını düz int'e dönüştür
    etiket_sayilari = {str(int(u)) + ' sayisi': int(c) for u, c in zip(unique, counts)}
    
    return alt_kume, etiket_sayilari

## Veri kümeleri indirme

In [ ]:
dataset_name = "turkish-nlp-suite/TrGLUE"
sst2_name = "sst2"
cola_name = "cola"
tohum_degeri = 571

In [ ]:
sst2_dataset = veri_kumesi_yukle(dataset_name, sst2_name)
cola_dataset = veri_kumesi_yukle(dataset_name, cola_name)
print(f"Veri kümeleri yüklendi.")

## Veri kümelerini ayrıştırma

In [ ]:
print("sst2 veri kümesi parçalanıyor...")
sst2_egitim, sst2_sinama, sst2_dict = veri_kumesi_parcala(sst2_dataset, egitim_boyutu=5000, sinama_boyutu=1000, tohum_degeri=tohum_degeri)
print("cola veri kümesi parçalanıyor...")
cola_egitim, cola_sinama, cola_dict = veri_kumesi_parcala(cola_dataset, egitim_boyutu=5000, tohum_degeri=tohum_degeri)
print("Veri kümeleri parçalandı.")
print(f"Veri kümesi boyutları:\nSST-2: Eğitim: {len(sst2_egitim)}, Sinama: {len(sst2_sinama)}\nCoLA: Eğitim: {len(cola_egitim)}, Sinama: {len(cola_sinama)}")
print(f"\nsst2 eğitim kümesi:\n{sst2_egitim[0:5]}\n\nsst2 sinama kümesi:\n{sst2_sinama[0:5]}\n\ncola eğitim kümesi:\n{cola_egitim[0:5]}\n\ncola sinama kümesi:\n{cola_sinama[0:5]}")

# Ayrıştırılan Veri Kümelerini Kaydetme

In [ ]:
def veri_kumesi_kaydet(veri_kumesi, dosya_adi):
    """
    Veri kümesini csv dosyası olarak kaydeder.
    
    Args:
    veri_kumesi (datasets.Dataset): Kaydedilecek veri kümesi
    dosya_adi (str): Kaydedilecek dosyanın adı
    """
    print(f"Veri kümesi kaydediliyor: {dosya_adi}")
    veri_kumesi_df = pd.DataFrame(veri_kumesi)
    dosya_yolu = os.path.join(veri_kumeleri_dizini, dosya_adi)
    veri_kumesi_df.to_csv(dosya_yolu, index=False)
    print(f"Veri kümesi kaydedildi: {dosya_yolu}")
import json
def veri_kumesi_dict_kaydet(veri_kumesi_dict, dosya_adi):
    """
    Veri kümesi bilgilerini csv dosyası olarak kaydeder.
    
    Args:
    veri_kumesi_dict (dict): Kaydedilecek veri kümesi bilgileri
    dosya_adi (str): Kaydedilecek dosyanın adı
    """
    print(f"Veri kümesi dict kaydediliyor: {dosya_adi}")
    veri_kumesi_dict_json = json.dumps(veri_kumesi_dict, indent=4)
    dosya_yolu = os.path.join(veri_kumeleri_dizini, dosya_adi)
    with open(dosya_yolu, 'w') as f:
        f.write(veri_kumesi_dict_json)
    print(f"Veri kümesi dict kaydedildi: {dosya_yolu}")

In [ ]:
veri_kumesi_kaydet(sst2_egitim, "sst2_egitim.csv")
veri_kumesi_kaydet(sst2_sinama, "sst2_sinama.csv")
veri_kumesi_kaydet(cola_egitim, "cola_egitim.csv")
veri_kumesi_kaydet(cola_sinama, "cola_sinama.csv")
veri_kumesi_dict_kaydet(sst2_dict, "sst2_dict.json")
veri_kumesi_dict_kaydet(cola_dict, "cola_dict.json")
print("TÜm veri kümeleri kaydedildi.")